## Task 2

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from glob import glob
import os

In [2]:
# This code adjusts the figure size of seaborn plots to 10 inches by 6 inches
sns.set(rc={'figure.figsize':(10,6)})

Creating a list called "topics" and populating
it with directories from the "Reuters_Dataset/training" folder
that contain fewer than or equal to 10 files.

In [3]:
''' 
This code creates a list called "topics" and populates
it with directories from the "Reuters_Dataset/training" folder
that contain fewer than or equal to 10 files.
'''
topics = []
for subset in glob('Reuters_Dataset/training/*'):
    topic = os.listdir(subset)
    # if len(topic)<=10:  
    topics.append(subset)

In [4]:
len(subset)

29

In [5]:
len(topics)

91

In [6]:
'''
This code reads text files from the directories specified in the "topics" 
list and appends their content to "train_corpus" while extracting labels from
the directory names and appending them to "train_labels".
'''
train_corpus = []
train_labels = []

for topic in topics:
    for file in glob(topic+'/*'):
        data = open(file)
        text = data.read()
        train_corpus.append(text)
        train_labels.append(topic.split('\\')[1])

In [7]:
# train_corpus

["\n\nCOMPUTER TERMINAL SYSTEMS <CPML> COMPLETES SALE\n\n     COMMACK, N.Y., Feb 26 - Computer Terminal Systems Inc said\nit has completed the sale of 200,000 shares of its common\nstock, and warrants to acquire an additional one mln shares, to\n<Sedio N.V.> of Lugano, Switzerland for 50,000 dlrs.\n    The company said the warrants are exercisable for five\nyears at a purchase price of .125 dlrs per share.\n    Computer Terminal said Sedio also has the right to buy\nadditional shares and increase its total holdings up to 40 pct\nof the Computer Terminal's outstanding common stock under\ncertain circumstances involving change of control at the\ncompany.\n    The company said if the conditions occur the warrants would\nbe exercisable at a price equal to 75 pct of its common stock's\nmarket price at the time, not to exceed 1.50 dlrs per share.\n    Computer Terminal also said it sold the technolgy rights to\nits Dot Matrix impact technology, including any future\nimprovements, to <Woodco 

In [8]:
train_labels

['acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',
 'acq',


In [9]:
'''
This code reads text files from the directories specified in the "topics"
list (with "training" replaced by "test") and appends their content to 
"test_corpus". It also extracts labels from the directory names and appends
them to "test_labels".
'''
test_corpus = []
test_labels = []

for topic in topics:
    topic = re.sub(r'training','test',topic)
    for file in glob(topic+'/*'):
        data = open(file)
        text = data.read()
        test_corpus.append(text)
        test_labels.append(topic.split('\\')[1])

# Cleaning

In [10]:
'''
This code defines a function called "clean_text" that performs various text cleaning operations.
It converts the text to lowercase, replaces underscores with spaces, removes mentions
(starting with "@"), removes hashtags (keeping only the text after "#"), expands contractions
(e.g., "he's" to "he is"), replaces special characters and punctuation with spaces, removes digits,
removes single-letter words, removes excessive whitespace, and finally, strips any leading or
trailing whitespace before returning the cleaned text.
'''
def clean_text(text):
    text = text.lower()
    text = re.sub(r'_', ' ', text)
    text = re.sub(r'@[A-Za-z0-9]+', ' ',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = re.sub(r"i'm", 'i am', text)
    text = re.sub(r"he's", 'he is', text)
    text = re.sub(r"she's", 'she is', text)
    text = re.sub(r"what's", 'what is', text)
    text = re.sub(r"where's", 'where is', text)
    text = re.sub(r"\'ll", 'will', text)
    text = re.sub(r"\'ve", 'have', text)
    text = re.sub(r"\'re", 'are', text)
    text = re.sub(r"\'d", 'had', text)
    text = re.sub(r"\n't", 'not', text)
    text = re.sub(r"[-()\"#/@;:<>`{}+=~|.!?,]", ' ', text)
    text = re.sub(r'[^\w\s]',' ', text)
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'^[a-z]\s+', ' ', text)
    text = re.sub(r'\s+[a-z]\s+',' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [11]:
'''
Function called "remove_stopwords" that takes a text input and removes the stopwords
(common words that typically do not add significant meaning to the text) using the NLTK library.
It tokenizes the text into words, filters out the stopwords, joins the remaining words into
a string, and returns the result.
'''
def remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    networds = [word for word in tokens if word not in stopwords.words('english')]
    networds = ' '.join(networds)
    return networds

'''
function called "lemmatize_text" that uses the WordNetLemmatizer from the NLTK library
to lemmatize the words in a given text. It tokenizes the text into words, applies lemmatization
to each word, joins the lemmatized words into a string, and returns the result.
'''
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    networds = [lemmatizer.lemmatize(word) for word in tokens]
    networds = ' '.join(networds)
    return networds

In [12]:
'''
This code iterates over the elements of the "train_corpus" and "test_corpus" lists,
applies a series of text cleaning operations (cleaning, removing stopwords, and lemmatizing)
to each text element, and then updates the corresponding element in the list with the cleaned and processed text.
'''
for e, txt in enumerate(train_corpus):
    txt = clean_text(txt)
    txt = remove_stopwords(txt)
    txt = lemmatize_text(txt)
    train_corpus[e] = txt
    
for e, txt in enumerate(test_corpus):
    txt = clean_text(txt)
    txt = remove_stopwords(txt)
    txt = lemmatize_text(txt)
    test_corpus[e] = txt

## Statistics

### Word Occurence

In [13]:
'''
This code creates a CountVectorizer object with a maximum document frequency of 1
(meaning all terms will be considered) and fits it on the combined "train_corpus"
and "test_corpus" texts. It transforms the text data into a matrix of token counts
(X) and retrieves the feature names (words) using the vectorizer's "get_feature_names()" method.
'''
vectorizer = CountVectorizer(max_df=1)
X = vectorizer.fit_transform(train_corpus+test_corpus)
feature_names = vectorizer.get_feature_names()

H:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
'''
This code initializes an empty pandas DataFrame called "word_stats" with columns for "word",
"corpus occurrence", and "document occurrence". It then iterates over each target word in the
feature names (words extracted from the corpus). For each target word, it finds the occurrence
of that word in each document, identifies the indices of the documents containing the word,
calculates the sum of occurrences across the corpus, and determines the total number of documents
containing the word. Finally, it adds a new row to the "word_stats" DataFrame with the
information for the target word.
'''
word_stats = pd.DataFrame(columns=['word','corpus occurence','document occurence'])
for target_word in feature_names:
    # Find the index of the target word in the feature names
    target_word_index = feature_names.index(target_word)

    # Get the occurrence of the target word in each document
    word_occurrence = X[:, target_word_index].toarray().flatten()

    # Find the indices of the documents that contain the target word
    documents_containing_word = [i for i, occurrence in enumerate(word_occurrence) if occurrence > 0]

    # Calculate the sum of occurrences
    total_occurrences = sum(word_occurrence)

    # Calculate the total number of documents containing the word
    total_documents_containing_word = len(documents_containing_word)
    
    word_stats.loc[len(word_stats)] = [target_word, total_occurrences, total_documents_containing_word]

In [15]:
'''
displays the first 10 rows of the "word_stats" DataFrame, showing
information about the word occurrences in the corpus.
'''
word_stats.head(10)

,word,corpus occurence,document occurence
0,aabex,1,1
1,aachener,1,1
2,aagiy,1,1
3,aaica,1,1
4,aaix,1,1
5,aap,2,1
6,aare,1,1
7,aarn,1,1
8,aarnoud,1,1
9,aart,1,1


# Clustering

In [16]:
from sklearn.cluster import KMeans

In [17]:
'''
perform K-means clustering on the data represented by the matrix X. It specifies the number
of clusters to create as 10 and initializes a KMeans object with the specified number of
clusters and a random state of 42. The K-means algorithm is then fit to the data using
the fit() method. The resulting cluster labels for each document are obtained using the
labels_ attribute, and the cluster centers are retrieved using the cluster_centers_ attribute.
'''
num_clusters = 10  # Number of clusters to create
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Get the cluster labels for each document
cluster_labels = kmeans.labels_

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

In [18]:
'''
This code iterates over each cluster generated by K-means clustering and performs the following steps:
Retrieves the indices of documents belonging to the current cluster.
Retrieves the actual document texts for the current cluster.
Calculates the term frequencies for the documents in the cluster.
Determines the most frequent word in the cluster based on the term frequencies.
Prints information about the cluster, including the most frequent word.
Stores the cluster documents corresponding to the most frequent word in a dictionary called "termsets_documents".
The code provides an overview of the clusters, their most frequent words, and the associated documents within each cluster.
'''

corpus = train_corpus+test_corpus
termsets_documents = {}

# Iterate over each cluster
for cluster_id in range(num_clusters):
    # Get the indices of documents in the current cluster
    cluster_indices = [i for i, label in enumerate(cluster_labels) if label == cluster_id]
    
    # Get the document texts in the current cluster
    cluster_documents = [corpus[i] for i in cluster_indices]
    
    # Calculate the term frequencies for the documents in the cluster
    cluster_term_frequencies = X[cluster_indices].toarray().sum(axis=0)
    
    # Find the most frequent word in the cluster
    most_frequent_word_index = cluster_term_frequencies.argmax()
    most_frequent_word = feature_names[most_frequent_word_index]
    
    # Print the cluster information
    print("Cluster {}: Most Frequent Word: '{}'".format(cluster_id+1, most_frequent_word))
    termsets_documents[most_frequent_word] = cluster_documents
    print()

Cluster 1: Most Frequent Word: 'lex'

Cluster 2: Most Frequent Word: 'jauppi'

Cluster 3: Most Frequent Word: 'guinn'

Cluster 4: Most Frequent Word: 'fermenta'

Cluster 5: Most Frequent Word: 'haughey'

Cluster 6: Most Frequent Word: 'tre'

Cluster 7: Most Frequent Word: 'audi'

Cluster 8: Most Frequent Word: 'cornelissen'

Cluster 9: Most Frequent Word: 'azraai'

Cluster 10: Most Frequent Word: 'antonson'



In [20]:
# show only 5 documents of random most frequent word
termsets_documents['lex'][:5]

['computer terminal system cpml completes sale commack feb computer terminal system inc said completed sale share common stock warrant acquire additional one mln share sedio v lugano switzerland dlrs company said warrant exercisable five year purchase price dlrs per share computer terminal said sedio also right buy additional share increase total holding pct computer terminal outstanding common stock certain circumstance involving change control company company said condition occur warrant would exercisable price equal pct common stock market price time exceed dlrs per share computer terminal also said sold technolgy right dot matrix impact technology including future improvement woodco inc houston tex dlrs said would continue exclusive worldwide licensee technology woodco company said move part reorganization plan would help pay current operation cost ensure product delivery computer terminal make computer generated label form tag ticket printer terminal',
 'ohio mattress omt may lowe